In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')

In [5]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df.columns = ['category', 'email']
df['label']= df.category.map(dict(ham=0,spam=1))
df.head()

,category,email,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [7]:
X=df.email
y=df.label

In [8]:
pd.value_counts(y).head(1)/len(y)

0    0.865937
Name: label, dtype: float64

In [9]:
import string

In [10]:
pun=string.punctuation

In [11]:
import spacy

In [12]:
nlp=spacy.load('en_core_web_sm')


In [13]:
from spacy.lang.en.stop_words import STOP_WORDS

In [14]:
stop_words= list(STOP_WORDS)

In [15]:
def clean_data(text):
    from spacy.lang.en.stop_words import STOP_WORDS
    doc=nlp(text)
    
    # create an emplty list of tokens
    tokens=[]
    # if token is not PRON then convert it to lower case and strip it to root word
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp=token.lemma_.lower().strip() 
    # if token is PRON then convert it to lower case
        else:
            temp=token.lower_ 
        tokens.append(temp)
   
    # Create an empty list of final cleaned tokens
    final_cleaned_tokens=[]
    
    # from the tokens extracted from earlier function remove all punctuation and stop words
    for token in tokens:
        if token not in stop_words and token not in pun:
            final_cleaned_tokens.append(token)
    return final_cleaned_tokens

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [17]:
tfidf= TfidfVectorizer(tokenizer=clean_data)

In [18]:
model= SVC(kernel='linear')

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [20]:
pipe = Pipeline([('tfidf',tfidf),('classifier',model)])

In [21]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function clean_data at 0x00000251F0C31678>)),
                ('classifier', SVC(kernel='linear'))])

In [22]:
y_pred= pipe.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
accuracy_score(y_test,y_pred)

0.9815116911364872

In [25]:
pipe.predict(['Offer is free'])

array([1], dtype=int64)

In [26]:
pipe.predict(["PFA the relevant docs"]) 

array([0], dtype=int64)